Import Everything


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import cv2
from cv2 import imread
import pandas as pd
import os
import matplotlib.pyplot as plt
#from tensorflow.keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.layers import Dropout, Activation
from tensorflow.keras.layers import Conv2D,BatchNormalization,MaxPool2D,Flatten,Dense
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Input, Activation, Dropout, GlobalAveragePooling2D, \
    BatchNormalization, concatenate, AveragePooling2D
from tensorflow.keras.optimizers import Adam
import time
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.applications import imagenet_utils
from imblearn.over_sampling import RandomOverSampler

Import Data

In [ ]:
X = np.load('/content/drive/MyDrive/Small Data SL/X_small_new.npy')
Y = np.load('/content/drive/MyDrive/Small Data SL/Y_small_new.npy')
print('shape of X',X.shape)
print('shape of Y',Y.shape)

shape of X (5942, 100, 100, 3)
shape of Y (5942,)


In [ ]:
X_new = X.reshape(X.shape[0],X.shape[1]*X.shape[2]*X.shape[3])
print(X_new.shape)

(5942, 30000)


In [ ]:
print(Y)
Y_new = np.where(Y==5,0,Y)
print(Y_new)
Y_new = np.where(Y_new==6,1,Y_new)
print(Y_new)
Y_new = np.where(Y_new==7,2,Y_new)
print(Y_new)
Y_new = np.where(Y_new==8,3,Y_new)
print(Y_new)

[5 5 5 ... 7 6 6]
[0 0 0 ... 7 6 6]
[0 0 0 ... 7 1 1]
[0 0 0 ... 2 1 1]
[0 0 0 ... 2 1 1]


In [ ]:
over_sample = RandomOverSampler()
X_new,Y_new = over_sample.fit_resample(X_new,Y_new)
X_again = np.array(X_new).reshape(-1,100,100,3)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Model

In [ ]:
model = Sequential()
model.add(Conv2D(16, kernel_size = (3,3), input_shape = (100, 100, 3), activation = 'relu', padding = 'same'))
model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', padding = 'same'))
model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))
model.summary()
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 100, 100, 16)      448       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 98, 98, 32)        4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 49, 49, 32)        9248      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 47, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 36864)            

Training the model without normalization(file name = best_model_wn.h5)

In [ ]:
X_again = (X_again-np.mean(X_again))/np.std(X_again) ## Here std(X) means RMS(root mean square), This step will decrease the volume of numbers and hence reduce the time taken
X_train, X_test, Y_train, Y_test = train_test_split( X_again , Y_new , test_size= 0.15, random_state=69 ) ### This step will shuffle and break the data into train and test

In [ ]:
callback = tf.keras.callbacks.ModelCheckpoint(filepath='best_model_new.h5',
                                                  monitor='val_acc', mode='max',
                                                 verbose=1)
model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer = 'adam',
              metrics = ['accuracy'])
history = model.fit(X_train,
                    Y_train,
                    validation_split=0.2,
                    batch_size = 128,
                    epochs = 20,
                    callbacks=[callback])

Epoch 1/20
37/37 [==============================] - 137s 4s/step - loss: 1.2672 - accuracy: 0.3940 - val_loss: 1.0859 - val_accuracy: 0.5267

Epoch 00001: saving model to best_model_new.h5
Epoch 2/20
37/37 [==============================] - 134s 4s/step - loss: 1.0233 - accuracy: 0.5344 - val_loss: 0.9377 - val_accuracy: 0.6014

Epoch 00002: saving model to best_model_new.h5
Epoch 3/20
37/37 [==============================] - 134s 4s/step - loss: 0.8654 - accuracy: 0.6219 - val_loss: 0.8058 - val_accuracy: 0.6658

Epoch 00003: saving model to best_model_new.h5
Epoch 4/20
37/37 [==============================] - 134s 4s/step - loss: 0.7048 - accuracy: 0.7135 - val_loss: 0.6858 - val_accuracy: 0.7226

Epoch 00004: saving model to best_model_new.h5
Epoch 5/20
37/37 [==============================] - 134s 4s/step - loss: 0.5651 - accuracy: 0.7769 - val_loss: 0.6389 - val_accuracy: 0.7379

Epoch 00005: saving model to best_model_new.h5
Epoch 6/20
37/37 [==============================] - 134